In [1]:
# Computes SC^2 using SOS for continuous time stochastic systems in verifying safety specifications

# include important libraries
using JuMP
using SCS # Splitting Conic Solver; Optimizer
using DynamicPolynomials
using MultivariatePolynomials
using LinearAlgebra
using TSSOS # important for SOS, see https://github.com/wangjie212/TSSOS
using Distributions # for the noise

### Stochastic Closure Certificate

In [2]:
@polyvar x1 x2 y1 y2 # global vars used in monomials
vars = [x1, x2, y1, y2]
y = [y1, y2]
n = 2 # dimension of ct-SS

sos_tol = 2 # the maximum degree of unknown SOS polynomials = deg + sos_tol 
error = 2   # precision digit places
γ = 8
λ = 143
ϵ = 1e-5

# ct-SS expressed as SDE dx(t) = f(x)dt + g(x)dW_t, where x=(x1, x2)
# drift f(.)
f_y = [
    y2,
    -y1 - y2 - 0.5*y1^3
]

# diffusion matrix g(.) 
g_xy = [
    0  0;
    0  0.1
]

# creates the parametrized SCC
function add_paramC_poly!(model, var1, deg)
    basis = monomials(var1, 0:deg) # basis in x and y
    coeffs = @variable(model, [1:length(basis)], base_name="c")
    CC = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return CC, coeffs, basis
end

# the domain in X
# poly describing X0 x Xu = {0.1^2-(x1+2)^2-x2^2>=0} x {x2-2.25>=0}
g0u = [0.1^2-(x1+2)^2-x2^2, y2-2.25]
g0 = [0.1^2-(x1+2)^2-x2^2] # initial set X0 = {0.1^2-(x1+2)^2-x2^2}
gX = [x1+3, 3-x1, x2+3, 3-x2, x1^2+x2^2-0.5^2, y1+3, 3-y1, y2+3, 3-y2, y1^2+y2^2-0.5^2] # state set X = [-3, 3]^2 U {x1^2 + x2^2 >= 0.5^2}


# polynomial stochastic closure certificate of degree deg over safety property
function scc(deg)
    model = Model(SCS.Optimizer)
    # set_optimizer_attribute
    set_optimizer_attributes(model,
        "eps_abs" => 1e-2,      # absolute tolerance
        "eps_rel" => 1e-2,      # relative tolerance
        "max_iters" => 500000,  # max iterations
        "alpha" => 1.5,         # step size parameter
        "normalize" => true,    # whether to rescale data
        "verbose" => 0          # silence solver output
    )

    C_dict = Dict()  # key => (symbolic_poly, numeric_poly)
    C, Cc, Cb = add_paramC_poly!(model, vars, deg) # CC, CC coefficients, and CC basis functions
    C_dict[1] = (C, Cc, Cb)  # store symbolic, coefficients, basis
    
    # non-negative CC
    add_psatz!(model, C, vars, gX, [], div(deg, 2); QUIET=true, CS=true, GroebnerBasis=false)

    # condition over X0 x Xu
    add_psatz!(model, C - λ - ϵ, vars, g0u, [], div(deg, 2); QUIET=true, CS=true, GroebnerBasis=false)
    
    # condition over X0 x X0
    # first modify C 
    EC = C(vars=>[x1,x2,x1,x2]) 
    # see TSSOS https://github.com/wangjie212/TSSOS
    add_psatz!(model, -EC + γ, [x1,x2], g0, [], div(deg, 2); QUIET=true, CS=true, GroebnerBasis=false)

    # condition over X x X
    # First term: ∇_y C . f_y
    grad_term = sum(differentiate(C, y[i]) * f_y[i] for i in 1:n)
    
    # Second term: 0.5 * tr(g g' ∇²_y C)
    # ito_term = 0.5 * sum(
    #     sum(g_xy[i, k] * g_xy[j, k] * differentiate(differentiate(C, y[i]), y[j])
    #         for k in 1:n) for i in 1:n, j in 1:n
    # )
    H = [differentiate(differentiate(C, y[i]), y[j]) for i in 1:n, j in 1:n] 
    ito_term = 0.5*0.1^2 * sum(H[i,i] for i in 1:n)
 
    # Complete stochastic Lie derivative wrt y (i.e infinitesimal generator)
    LyC = grad_term + ito_term
    
    add_psatz!(model, -LyC, vars, gX, [], div(deg, 2); QUIET=true, CS=true, GroebnerBasis=false)
        
    optimize!(model) # solve for coefficients
    status = termination_status(model)
    pri = primal_status(model)
    dua = dual_status(model)
    C_eval_dict = Dict() # Get numerical values of coefficients and plug into polynomials
    for (key, (C, Cc, Cb)) in C_dict
        coeff_vals = round.(value.(Cc); digits=error)  # Round each coefficient to 2 decimal places
        C_numeric = sum(coeff_vals[i] * Cb[i] for i in eachindex(Cb))
        C_eval_dict[key] = (C, C_numeric)
    end
    
    # status might be optimal but if all Bc approx 10^{-error}, it's essentially 0 so OPTIMAL != CC.
    p_sat = 1-γ/λ # satisfaction probability lower bound
    return status, C_eval_dict, p_sat, pri, dua
end
        
# Simulation
deg = 6 # desired degree of SCC
stats = @timed (status, CC_data, p_sat, pri, dua) = scc(deg)

println("poly deg: ", deg)
println("status: ", status)
println("status: ", pri)
println("status: ", dua)
println("sat probability: ", p_sat)
println("Number of SCC polynomials: ", length(CC_data))
println("time: ", stats.time, "\n")

# Print the dictionary line-by-line
for (k, v) in CC_data
    println("Key: ", k)
    println("Polynomial: ", v[1])
    println("Coefficients: ", v[2], "\n")
end

println("Finished")

poly deg: 6
status: OPTIMAL
status: FEASIBLE_POINT
status: FEASIBLE_POINT
sat probability: 0.9440559440559441
Number of SCC polynomials: 1
time: 38.93282925

Key: 1
Polynomial: (c[1]) + (c[2])*y2 + (c[3])*y1 + (c[4])*x2 + (c[5])*x1 + (c[6])*y2^2 + (c[7])*y1*y2 + (c[8])*y1^2 + (c[9])*x2*y2 + (c[10])*x2*y1 + (c[11])*x2^2 + (c[12])*x1*y2 + (c[13])*x1*y1 + (c[14])*x1*x2 + (c[15])*x1^2 + (c[16])*y2^3 + (c[17])*y1*y2^2 + (c[18])*y1^2*y2 + (c[19])*y1^3 + (c[20])*x2*y2^2 + (c[21])*x2*y1*y2 + (c[22])*x2*y1^2 + (c[23])*x2^2*y2 + (c[24])*x2^2*y1 + (c[25])*x2^3 + (c[26])*x1*y2^2 + (c[27])*x1*y1*y2 + (c[28])*x1*y1^2 + (c[29])*x1*x2*y2 + (c[30])*x1*x2*y1 + (c[31])*x1*x2^2 + (c[32])*x1^2*y2 + (c[33])*x1^2*y1 + (c[34])*x1^2*x2 + (c[35])*x1^3 + (c[36])*y2^4 + (c[37])*y1*y2^3 + (c[38])*y1^2*y2^2 + (c[39])*y1^3*y2 + (c[40])*y1^4 + (c[41])*x2*y2^3 + (c[42])*x2*y1*y2^2 + (c[43])*x2*y1^2*y2 + (c[44])*x2*y1^3 + (c[45])*x2^2*y2^2 + (c[46])*x2^2*y1*y2 + (c[47])*x2^2*y1^2 + (c[48])*x2^3*y2 + (c[49])*x2^3*y1 + (